In [1]:
import requests 
import time
import datetime as dt
import pandas as pd

In [18]:
api_url = 'https://data.lacounty.gov/resource/mk7y-hq5p.json'
assessor_json = requests.get(api_url).json()


In [19]:
assessor_keys = assessor_json[0].keys()
assessor_keys

dict_keys(['cluster', 'situsdirection', 'roll_landbaseyear', 'roll_totalvalue', 'situszip', 'situshouseno', 'sqftmain', 'roll_perspropexemp', 'adminregion', 'rowid', 'situszip5', 'roll_landvalue', 'roll_fixturevalue', 'roll_homeownersexemp', 'situsunit', 'taxratearea', 'center_lon', 'nettaxablevalue', 'bathrooms', 'bedrooms', 'rollyear', 'recordingdate', 'usecode', 'location_1', 'parcelclassification', 'ain', 'units', 'roll_impbaseyear', 'roll_impvalue', 'taxratearea_city', 'roll_totlandimp', 'roll_realestateexemp', 'parcelboundarydescription', 'totbuildingdatalines', 'situsfraction', 'propertylocation', 'roll_perspropvalue', 'istaxableparcel', 'yearbuilt', 'situscity', 'center_lat', 'roll_totalexemption', 'roll_fixtureexemp', 'assessorid', 'usecodedescchar4', 'situsstreet', 'usecodedescchar3', 'usecodedescchar2', 'usecodedescchar1', 'effectiveyearbuilt', 'usetype'])

In [20]:
len(assessor_json)

1000